##### %%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

In [ ]:
# Write Here All your Names, Surnames, and Student ID (Codice Persona)

e.g.

[
    "Mario", "Beraha", "123431451",
    "Matteo", "Gianella", "913931"

]

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

np.random.seed(0)

# Mixture Modelling

## aka modelling heterogeneity

$\newcommand{\iid}{\stackrel{\mbox{iid}}{\sim}}$


In mixture model, we account for heterogeneity in the data by assuming that there is not "one single" data generating process, but actually K of them.

Therefore, we expect to find K different subpopulations (== clusters) in our data, and that each subpopulation is homogeneous: it is suitably modeled by a density, typically from a parametric family.

In its most general form, let $f_1(\cdot), \ldots, f_K(\cdot)$ be $K$ probability density functions over a space $\mathbb{Y}, \mathcal{Y}$, $\mathbf{w} = (w_1, \ldots, w_K)$ a vector in the $K$-simplex; a mixture model assumes the following likelihood

\begin{equation}
    y_1, \ldots, y_n \mid \mathbf{w}, \{f_j\}_{j=1}^K \iid f^*(y) := \sum_{j=1}^K w_j f_j(y)
    \tag{1}
\end{equation}

We refer to the $f_j$'s as the _components_ of the mixture

$\newcommand{\ind}{\stackrel{\small \mbox{ind}}{\sim}}$

The connection to clustering is made more explicit by introducing "cluster assignment" variables $c_i$, $i=1, \ldots, n$ such that
$$
    P(c_i = h \mid \mathbf{w}) = w_h
$$

Then, the mixture likelihood can be restated as the following hierarchical model

\begin{equation}
    \begin{aligned}
        y_i \mid c_i = h, \{f_j\}_{j=1}^K & \ind f_h \\
        P(c_i = h \mid \mathbf{w}) = w_h
    \end{aligned}
    \tag{2}
\end{equation}

Usually, $\{f_j\}_{j=1}^K$ belong to the same parametric family (e.g., the normal distribution), and differ only for the specific values of the parameters (e.g., every component has a different mean/variance). But this might not always be the case.

# Exercise 1: Gaussian Mixture Model

Consider the dataset in the file "gestational_age_data.csv", that contains the gestational age of more than two thousand children (in days).

We can model the data using a mixture of three gaussian distributions.

\begin{equation*}
    y_1, \ldots, y_n \mid \mathbf{w}, \mathbf{\mu}, \mathbf{\sigma}^2 \iid \sum_{h=1}^5 w_h \mathcal{N}(\mu_h, \sigma^2_h) 
\end{equation*}


1.1) Describe a simple prior for $\mathbf{w}$

1.2) Using the likelihood in (2), derive the full conditional for $\mathbf{w}$

1.3) What is a simple semi-conjugate prior for $(\mu_h, \sigma_h^2)$. Derive the full conditional for these parameters

1.4) Describe a Gibbs sampler that alternates between updating the $c_i$'s, the weights $\mathbf w$ and the "unique values" $(\mu_1, \sigma_1^2), \ldots (\mu_K, \sigma_K^2)$ where in this example $K=5$.


1.5) Complete the model by specifying a prior such that the expected value of each $\mu_h$ is equal to the empirical/sample mean and the expected value of each $\sigma^2_h$ is equal to one half of the sample variance. Choose the remaining parameters "wisely".

## Answer

1.1)

1.2)

1.3)

1.4)

1.5)

In [ ]:
dataset = pd.read_csv("songs_data.csv")
dataset.head()

In [ ]:
data = dataset.acousticness.to_numpy()
np.random.shuffle(data)
sns.histplot(data)
plt.show()

In [ ]:
def run_one_gibbs(data, cluster_allocs, uniq_vals, weights):
    """
    Runs one iteration of the gibbs sampler
    Parameters
    ----------
    data: a numpy array of floats, contains the observations
    cluster_allocs: a numpy array of integers, contains the c_i's
        (same length of data)
    uniq_vals: a numpy array of size [K, 2], contains (mean, variance)
        for every component
    weights: a numpy array of size [K], contains the weights of the
        components
    """
    
    n_clus = len(weights)
    
    for h in range(n_clus):
        clusdata = data[cluster_allocs == h]
        if len(clusdata) == 0:
            uniq_vals[h, :] = sample_uniq_vals_prior()
        else:
            uniq_vals[h, :] = sample_uniq_vals_fullcond(
                clusdata, uniq_vals[h, 0], uniq_vals[h, 1])
    
    cluster_allocs = update_cluster_allocs(data, weights, uniq_vals)
    
    weights = update_weights(cluster_allocs, n_clus)
    
    return cluster_allocs, uniq_vals, weights



def run_mcmc(data, niter=7500, nburn=2500, thin=5, n_clus=5):
    cluster_allocs = tfd.Categorical(probs=np.ones(n_clus) / n_clus).sample(len(data))
    weights = np.ones(n_clus) / n_clus
    
    # TODO Complete by sampling from an appropriate distribution
    uniq_vals = np.dstack([
        tfd.VARIABLE1(??, ??).sample(n_clus),
        tfd.VARIABLE2(??, ??).sample(n_clus)])[0, :, :]
    
    allocs_out = []
    uniq_vals_out = []
    weights_out = []
    
    for i in range(niter):
        cluster_allocs, uniq_vals, weights = run_one_gibbs(
            data, cluster_allocs, uniq_vals, weights)
        
        if i > nburn and i % thin == 0:
            allocs_out.append(cluster_allocs)
            uniq_vals_out.append(uniq_vals)
            weights_out.append(weights)
            
        if i % 10 == 0:
            print("\rIter {0} / {1}".format(i+1, niter), flush=True, end=" ")
            
    return allocs_out, uniq_vals_out, weights_out

In [ ]:
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions


def update_cluster_allocs(data, weights, uniq_vals):
    # Hint: the update of each c_i can be done independently of
    # the other c_j's (j != i) ---> Use numpy broadcasting /vectorizaiton
    # to speed up the code
    # Hint: you might need to evaluate the likelihood of each observation
    # with parameters in each cluster. The following snippet returns a
    # [n, K] matrix with [i, j]-entry equal to log_prob(y_i | \mu_j, \sigma^2_j)
    
    logprobs = tfd.Normal(
        uniq_vals[:, 0], np.sqrt(uniq_vals[:, 1])).log_prob(data[:, np.newaxis])
    
    # TODO: probs should be a [n, n_clus] matrix with [i, j]-entry equal to
    # P(c_i = j | all the rest)
    probs = None
    return tfd.Categorical(probs=probs).sample()


def update_weights(cluster_allocs, n_clus):
    # Hint: you might need to count how many observations are in each cluster
    # you can use the following code
    # n_by_clus = np.array([np.sum(cluster_allocs == h) for h in range(n_clus)])

    
    post_params = None
    return tfd.Dirichlet(post_params.astype(float)).sample()


def sample_uniq_vals_prior():
    # TODO: choose appropriate variables and parameters
    var = tfd.VARIABLE2(??, ??).sample()
    mu = tfd.VARIABLE1(??, ??).sample()
    return np.array([mu, var])
    

def sample_uniq_vals_fullcond(clusdata, mean_old, var_old):
    # TODO: compute the full conditional parameters
    var = tfd.VARIABLE2(??, ??).sample()
    mu = tfd.VARIABLE1(??, ??).sample()
    return np.array([mu, var])

In [ ]:
allocs_out, uniq_vals_out, weights_out = run_mcmc(data)

In [ ]:
plt.hist(data, density=True, alpha=0.1)
for h in range(5):
    currd = data[allocs_out[-1] == h]
    plt.scatter(currd, np.zeros_like(currd) + 0.1 * (h+1))

# Hint!!

If you want to do Exs 1 & 2 in parallel, you can simulate a fake Markov Chain that mimics the output of Ex1.

For $j=1, \ldots, M$ sample

$$
    \mu_1^{(j)} \sim \mathcal{N}(-5, 0.5), \quad \mu_2^{(j)} \sim \mathcal{N}(5, 0.5)
$$
$$
    \sigma^{2, (j)}_1 \sim \mathcal{IG}(3,3 ), \quad \sigma^{2, (j)}_2 \sim \mathcal{IG}(3,3 )
$$
$$
    c_i^{(j)} \sim \text{Categorical(0.9, 0.1)}, \text{for i =} 1, \ldots, 50 
$$
$$
    c_i^{(j)} \sim \text{Categorical(0.1, 0.9)}, \text{for i =} 51, \ldots, 100
$$
$$
    \mathbf{w}^{(j)} \sim \text{Dirichlet}(5, 5)
$$

In [27]:
M = 5
N_mezzi = 50
N = 2*N_mezzi

dist_mu_1 = tfd.Normal(loc=-5,scale=0.5)
dist_mu_2 = tfd.Normal(loc=5,scale=0.5)
mu_1_vec = dist_mu_1.sample(M)
mu_2_vec = dist_mu_2.sample(M)

dist_sigma_2_1 = tfd.InverseGamma(concentration = 3, scale = 3)
dist_sigma_2_2 = tfd.InverseGamma(concentration = 3, scale = 3)
sigma_2_1_vec = dist_sigma_2_1.sample(M)
sigma_2_2_vec = dist_sigma_2_2.sample(M)


c_i_j = np.concatenate((np.reshape(tfd.Categorical(probs=[0.9,0.1]).sample(N_mezzi*M),(M,N_mezzi)), np.reshape(tfd.Categorical(probs=[0.1,0.9]).sample(N_mezzi*M),(M,N_mezzi))), axis=1)

w_vec = tfd.Dirichlet([5,5]).sample(M)

# i dati sono
mu_1_vec, mu_2_vec, sigma_2_1_vec, sigma_2_2_vec, c_i_j, w_vec



(array([-5.325928 , -5.8873725, -5.2366805, -5.101785 , -4.7250714],
       dtype=float32),
 array([5.00045  , 4.228856 , 5.431074 , 4.9407067, 5.2441854],
       dtype=float32),
 array([1.1361796 , 5.7812686 , 0.36698982, 1.4152927 , 0.7762139 ],
       dtype=float32),
 array([2.0479698 , 0.88734555, 1.3755735 , 1.0936158 , 1.1971056 ],
       dtype=float32),
 array([[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 

# Exercise 2: Posterior inference in Mixture Models

Mixture models are subject to the so called "label switching", which causes non-identifiability for all the parameters.

Label switching means that permuting the "labels" {1, 2, ..., K} of the cluster yields the same exact likelihood.
Hence, it does not make sense to talk about "first", "second" and "third" cluster.
Moreover, also interpretation of the cluster parameters is tricky: we cannot say that

$$
    \frac{1}{M} \sum_{j=1}^M \mu^{(j)}_1
$$

is an estimate of parameter $\mu_1$. Also, it does not make sense to take averages of the cluster allocation labels.

There are several ways to solve this non-identifiability issue, we will focus here on a decision-theoretic approach.


2.1) Implement a function that finds the clustering, among the ones visited in the MCMC chain, that minimizes Binder's loss function:

$$
    \sum_{i=1}^n \sum_{j=1}^n (\mathbb I [c_i = c_j] - p_{ij})^2
$$
where $p_{ij}$ is the posterior probability that observations $i$ and $j$ belong to the same cluster.

Note that $p_{ij}$ can be easily estimated from the MCMC chains!

2.2) Given the estimated clustering $\mathbf{c}^*$, estimate the parameters in each cluster. Let $c^*_h$ denote the index-set of the $h$--th cluster (e.g., $c^*_1 = \{1, 3, 10\}$, $c^*_2 = \{0, 2, 4, 5, 6\}$ ...)
Then
$$
    \hat \mu_h = \frac{1}{M} \sum_{j=1}^M \frac{1}{\# c^*_h} \sum_{i \in c^*_h} \mu^{(j)}_{c_i}
$$
where $\mu^{(j)}_\ell$  is the mean parameter in the $\ell$--th cluster at the $j$--th MCMC iteration.

Similarly for the variance $\hat \sigma_h^2$.

2.3) Comment on the posterior inference obtained

In [ ]:
def get_psm(clus_alloc_chain):
    """
    Returns the posterior similarity matrix, i.e.
        out[i, j] = P(c_i == c_j | all the rest)
    for each pair of observations
    """
    c_chain = np.vstack(clus_alloc_chain)
    out = np.zeros((c_chain.shape[1], c_chain.shape[1]))
    # TODO: fill out!       
    return out


def minbinder_sample(clus_alloc_chain, psm):
    """
    Finds the iteration for which the Binder loss is minimized
    and returns the associated clustering
    """
    losses = np.zeros(len(clus_alloc_chain))
    c_chain = np.vstack(clus_alloc_chain)
    
    # You can either cycle through the iterations, or 
    # cycle through the entries in the PSM [i, j]
    # and vectorize the same operation for each iteration!
    
    # TODO: compute the losses!
   
    best_iter = np.argmin(losses)
    return clus_alloc_chain[best_iter]

In [ ]:
psm = get_psm(allocs_out)

In [ ]:
best_clus = minbinder_sample(allocs_out, psm)

In [ ]:
plt.hist(data, density=True, alpha=0.3)
for h in range(5):
    currd = data[best_clus == h]
    plt.scatter(currd, np.zeros_like(currd) + 0.1 * (h+1))

In [ ]:
def unique_vals_given_clus(unique_vals_chain, clus_alloc_chain, best_clus):
    c_allocs = np.stack(clus_alloc_chain)
    uniq_vals = np.stack(unique_vals_chain)
    means = uniq_vals[:, :, 0]
    variances = uniq_vals[:, :, 1]
    out = []
    
    for h in np.unique(best_clus):
        data_idx = np.where(best_clus == h)[0]
        uniq_vals_idx = c_allocs[:, data_idx] # -> Matrix [n_iter x n_data_in_clus]
        means_by_iter = np.empty((c_allocs.shape[0], len(data_idx)))
        vars_by_iter = np.empty_like(means_by_iter)
        for i in range(c_allocs.shape[0]):
            means_by_iter[i, :] = ???
            vars_by_iter[i, :] = ???

        avg_mean_by_iter = ???
        avg_var_by_iter = ???
        
        muhat = np.mean(avg_mean_by_iter)
        sigsqhat = np.mean(avg_var_by_iter)
        out.append(np.array([muhat, sigsqhat]))
        
    return out

In [ ]:
clus_vals = unique_vals_given_clus(uniq_vals_out, allocs_out, best_clus)
clus_vals

In [ ]:
weights = np.sum(best_clus == np.arange(5)[:, np.newaxis], axis=1) / len(data)

In [ ]:
plt.hist(data, density=True, alpha=0.1, bins=30)
xx = np.linspace(np.min(data), np.max(data), 1000)
out_dens = np.zeros_like(xx)
for i in range(5):
    comp_dens = tfd.Normal(clus_vals[i][0], np.sqrt(clus_vals[i][1])).prob(xx)
    out_dens += weights[i] * comp_dens
    plt.fill_between(xx, np.zeros_like(xx), weights[i] * comp_dens, alpha=0.6)

plt.plot(xx, out_dens, c="black", linewidth=3)
plt.show()

## Comment on the inference obtained

The three clusters differ in...

Their weights are ...

Hence we can conclude that the population ...

# Exercise 3: Latent Dirichlet Allocation

We now move to one of the most fundamental papers in Machine Learning / Text Mining: Latent Dirichlet Allocations by Blei et al (2003).

The goal is topic modelling given a corpus (= collection of documents)

Denote with $\mathbf y_i = \{y_{i1}, \ldots, y_{i N_i}\}$ the $i$--th document where each $y_{ij}$ denotes a word in a vocabulary, i.e. it is a categorical variable. Let $|V|$ be the size of the vocabulary (number of unique words) so that $y_{ij} \in \{1, 2, \ldots, |V|\}$, and $K$ the number of topics.

For each topic $k$, we assume a different distribution over the vocabulary $\beta_k = (\beta_{k,1}, \ldots \beta_{k, |V|})$, where the vector $\beta_k$ in the $|V|$-dimensional simplex encodes the probability of occurence of each word in topic $k$ (i.e., informally, $\beta_{k, j} = P(\text{"we use word j when talking about topic k"})$  

Moreover, in each document more than one topic can be discussed. Denote with $w_i = (w_{i, 1}, \ldots, w_{i, K})$ the weight of each topic in the $i$--th document.

Then LDA assumes

\begin{equation}
\begin{aligned}
    P(y_{i, j} = v \mid \{\beta_k\}_{k=1}^J, w_i) &= \sum_{k=1}^K w_{i, k} \beta_{k, v}, \quad v=1, \ldots, |V| \\
    \beta_k &\sim \text{Dirichlet}(\alpha_1, \ldots, \alpha_{|V|}) \\
    w_i & \sim \text{Dirichlet}(0.5, \ldots, 0.5)
\end{aligned}
\label{eq:LDA} \tag{3}
\end{equation}


3.1) Rewrite the likelihood introducing suitable auxiliary variables $t_{ij}$ so that $w_i$ does appear on the right hand side of the first equation in \eqref{eq:LDA}

3.2) Specify the hyperparameters of the prior for the $\beta_j$'s such that, a priori, the expected value of $\beta_{j, v}$ is proportional to the number of times word $v$ appears in the corpus. What about the variance?

3.3) Implement a Gibbs sampler to perform posterior inference. Instead of identifying the mixture parameters, consider only the last iteration of the MCMC. What can we say about the topics? 

### SOLUTIONS

3.1) We introduce $t_{ij} \sim \text{TODO}$ (one for each $y_{ij}$) so that
\begin{equation*}
\begin{aligned}
    P(y_{ij} = v \mid \{\beta_k\}, t_{ij} = h) &= \beta_{h, v} \\
    P(t_{ij} = h \mid \pi_i) &= \text{TODO}
\end{aligned}
\end{equation*}


3.2) We fix the expected value as ....

To control the variance, call $d_v =  \sum_{i=1}^D \sum_{j=1}^{N_i} \mathbb{I}[y_{ij} = v]$, so that $\alpha_v = \kappa d_v$.

3.3) The joint model is
\begin{equation*}
\begin{aligned}
    \mathcal{L}(\{y_{ij}\}, \{\beta_k\}, \{w_i\}, \{t_{ij}\}) &= \prod_{i=1}^D \prod_{j=1}^{N_i} w_{i, t_{ij}} \prod_{v=1}^{|V|} \beta_{t_{ij}, v}^{\mathbb{I}[y_{ij} = v]} \\
    & \times \prod_{k=1}^K \frac{1}{B(\mathbf \alpha)} \prod_{v=1}^{|V|} \beta_{k, v}^{\alpha_v - 1} \mathbb{I}[\beta_k \in S^{|V|}] \\
    & \times \prod_{i=1}^D \frac{1}{B(\mathbf{0.5})} \prod_{k=1}^{K} w_{i, k}^{0.5 - 1} \mathbb{I}[w_i \in S^{K}]
\end{aligned}
\end{equation*}

Then it is easy to see that

1)
$$
    \mathcal{L}(w_i \mid \cdots) \propto \prod_{k=1}^{K} \text{TODO}
$$

Hint: rewrite 

$$
    \prod_{j=1}^{N_i} w_{i, t_{ij}} = \prod_{k=1}^K w_{i, k}^{n_{i, k}}
$$
where $n_{i, k} = ???$.

Finally, conclude that $w_i \mid \cdots \sim \text{TODO}$


2) $$ \mathcal{L}(\beta_k \mid \cdots) \propto \text{TODO}
   $$
   
   Hence $\beta_k \mid \cdots \sim \text{TODO}$
   
   



3) If $y_{ij} = v^*$, 
$$
    P(t_{ij} = k \mid \cdots) \propto w_{ik} \beta_{k, v^*}, \quad k=1, \ldots, K
$$

Hence, $t_{ij}$ is categorically-distributed with probabilities proportional to $w_{ik} \beta_{k, v^*}$. We can sample $t_{ij}$ from a Categorical distribution after having computed the unnormalized probabilities for each $k$ and having normalized them.

In [ ]:
# !pip3 install scikit-learn

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
data = fetch_20newsgroups()["data"]

In [ ]:
data

In [ ]:
## Some preprocessing.
import re

def preprocess_text(text):
    "Removes digits and weird punctuation"
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\__', '', text)
    text = re.sub(r'\___', '', text)
    return text

vectorizer = CountVectorizer(
    preprocessor=preprocess_text,
    stop_words="english",
    strip_accents="unicode",
    max_df=0.95,
    min_df=0.01,
    max_features=500)

vectorizer.fit(data)
vectorizer.get_feature_names()[:50]

In [ ]:
analyzer = vectorizer.build_analyzer()
docs = []
for doc in data:
    tokens = analyzer(doc)
    idxs = np.array(list(filter(
        lambda x: x, 
        [vectorizer.vocabulary_.get(tok, None) for tok in tokens])))
    docs.append(idxs)    

## Docs is your final dataset, work only with it!

docs is a list where every entry represents a document. A document is represented as a list of indexes (each work represents a word)

In [ ]:
n_topics = 20
voc_len = len(vectorizer.get_feature_names())
alpha = TODO

In [ ]:
def lda_gibbs_sampler(docs):
    """
    Parameters
    ----------
    docs: a list of np.arrays of integers, each represnts the 
        words in a document
        
    word_topics are the t_{ij}'s
    voc_weights are the \beta_k's
    doc_weights are the w_i's
    """
    
    word_topics = [
        tfd.Categorical(probs=np.ones(n_topics)).sample(len(x)) for x in docs]
    voc_weights = tfd.Dirichlet(alpha).sample(n_topics)
    doc_weights = tfd.Dirichlet(np.ones(n_topics) * 0.5).sample(len(docs))
    
    for i in range(300):
        print("\r{0} / {1}".format(i+1, 300), flush=True, end=" ")
        word_topics, doc_weights, voc_weights = run_one_lda(
            docs, word_topics, doc_weights, voc_weights)
    return word_topics, doc_weights, voc_weights


def run_one_lda(docs, word_topics, doc_weights, voc_weights):    
    word_topics = update_word_topics(doc_weights, voc_weights, docs)
    doc_weights = update_doc_weights(word_topics)
    voc_weights = update_voc_weights(docs, word_topics)
    return word_topics, doc_weights, voc_weights


def update_voc_weights(docs, word_topics):
    """Updates \beta_k for every k"""
    out = np.zeros((n_topics, voc_len))
    for k in range(n_topics):
        # Get all the words whose topic is k (i.e. t_ij = k)
        # Kept words is a np.array of integers with the indexes/words
        # of all y_{ij} for which t_{ij} = k
        mask = [np.where(x == k) for x in word_topics]
        kept_words = np.concatenate([x[y] for x, y in zip(docs, mask)])
        
        # Returns the unique values in kept_words and their counts
        # For instance if kept_words = [1, 1, 1, 0, 5, 4]
        # --> idxs = [0, 1, 4, 5]
        # --> counts = [1, 3, 1, 1]
        idxs, counts = np.unique(kept_words, return_counts=True)
        
        dir_param = ???
        out[k] = tfd.DISTRIBUTION(dir_param).sample()
    return out


def update_doc_weights(word_topics):
    """Updates w_i for every i"""
    out = np.zeros((len(docs), n_topics))
    for doc_ind, word_topic_in_doc in enumerate(word_topics):
        # word_topic_in_doc is the doc_ind-th row of word_topics
        
        # cnt returns how many times each topic appears in each document
        # i.e. cnt[k] is the number of words ind the doc_ind-th document
        # that have topic k
        cnt = np.sum(
            word_topic_in_doc == np.arange(n_topics)[:, np.newaxis], axis=1)
        
        dir_param = ???
        out[doc_ind] = tfd.DISTRIBUTION(dir_param).sample()
    return out


def update_word_topics(doc_weights, voc_weights, docs):
    out = []
    for i, doc in enumerate(docs):
        # computes an [num_words x K] matrix with entries
        # probas[l, m] beta_{m, v_l} * w_{i, m}
        # where v_l is the word in position 'l'
        # --> for each word (by row) represents the unnormalized probability
        #     of each topic
        probas = doc_weights[i][np.newaxis, :] * voc_weights[:, doc].T
        
        # normalize it approriately
        probas = ???
        out.append(tfd.Categorical(probs=probas).sample())
        
    return out

In [ ]:
word_topics, doc_weights, voc_weights = lda_gibbs_sampler(docs[:5000])

In [ ]:
for k in range(n_topics):
    top_ind = np.argpartition(voc_weights[k, :], -10)[-10:]
    print("Topic: {0}".format(k))
    print(", ".join([vectorizer.get_feature_names()[ind] for ind in top_ind]))
    print("\n\n")